In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ._user-wallet-transactions.json to ._user-wallet-transactions (1).json


In [ ]:
import json
import pandas as pd
from datetime import datetime
from collections import defaultdict

def load_transactions(json_file_path):
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return data

def extract_features(transactions):
    wallets = defaultdict(list)
    for txn in transactions:
        if 'user' not in txn:
            continue
        wallets[txn['user']].append(txn)

    features = []
    for wallet, txns in wallets.items():
        deposit_sum = 0
        borrow_sum = 0
        repay_sum = 0
        redeem_sum = 0
        liquidation_count = 0
        tx_counts = defaultdict(int)
        tx_amounts = []
        timestamps = []

        for txn in txns:
            tx_type = txn.get('action', '').lower()
            amount = float(txn.get('amount', 0))
            timestamp = int(txn.get('timestamp', 0))
            tx_amounts.append(amount)
            timestamps.append(timestamp)
            tx_counts[tx_type] += 1

            if tx_type == 'deposit':
                deposit_sum += amount
            elif tx_type == 'borrow':
                borrow_sum += amount
            elif tx_type == 'repay':
                repay_sum += amount
            elif tx_type == 'redeemunderlying':
                redeem_sum += amount
            elif tx_type == 'liquidationcall':
                liquidation_count += 1

        tx_count = len(txns)
        active_days = len(set([datetime.utcfromtimestamp(ts).date() for ts in timestamps]))
        repay_to_borrow_ratio = repay_sum / borrow_sum if borrow_sum > 0 else 0
        avg_tx_amount = sum(tx_amounts) / tx_count if tx_count > 0 else 0

        timestamps_sorted = sorted(timestamps)
        time_diffs = [(timestamps_sorted[i] - timestamps_sorted[i-1]) for i in range(1, len(timestamps_sorted))]
        avg_time_diff = sum(time_diffs)/len(time_diffs) if len(time_diffs) > 0 else 0

        features.append({
            'wallet': wallet,
            'deposit_sum': deposit_sum,
            'borrow_sum': borrow_sum,
            'repay_sum': repay_sum,
            'redeem_sum': redeem_sum,
            'liquidation_count': liquidation_count,
            'tx_count': tx_count,
            'active_days': active_days,
            'repay_to_borrow_ratio': repay_to_borrow_ratio,
            'avg_tx_amount': avg_tx_amount,
            'avg_time_diff': avg_time_diff,
            'borrow_count': tx_counts['borrow'],
            'repay_count': tx_counts['repay'],
            'deposit_count': tx_counts['deposit'],
        })

    return pd.DataFrame(features)

def score_wallets(df):
    max_deposit = df['deposit_sum'].max() if df['deposit_sum'].max() > 0 else 1
    max_borrow = df['borrow_sum'].max() if df['borrow_sum'].max() > 0 else 1
    max_active_days = df['active_days'].max() if df['active_days'].max() > 0 else 1

    scores = []
    for _, row in df.iterrows():
        deposit_score = min(row['deposit_sum'] / max_deposit, 1) * 300
        repay_ratio_score = min(row['repay_to_borrow_ratio'], 1) * 300
        liquidation_penalty = min(row['liquidation_count'], 10) * 50
        active_days_score = min(row['active_days'] / max_active_days, 1) * 200
        tx_count_score = min(row['tx_count'] / 100, 1) * 100
        score = deposit_score + repay_ratio_score + active_days_score + tx_count_score - liquidation_penalty
        score = max(score, 0)
        score = min(score, 1000)
        scores.append({'wallet': row['wallet'], 'score': int(score)})
    return pd.DataFrame(scores)

def run_scoring(json_path):
    print(f"Loading {json_path}")
    txns = load_transactions(json_path)
    print(f"Total transactions: {len(txns)}")
    features = extract_features(txns)
    print(f"Total wallets: {len(features)}")
    scores = score_wallets(features)
    print("Scores calculated.")
    return scores


In [ ]:
with open("/content/user-wallet-transactions.json", "r") as f:
    txns = json.load(f)

print(txns[0])  # Print the first transaction


{'_id': {'$oid': '681d38fed63812d4655f571a'}, 'userWallet': '0x00000000001accfa9cef68cf5371a23025b6d4b6', 'network': 'polygon', 'protocol': 'aave_v2', 'txHash': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190', 'logId': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190_Deposit', 'timestamp': 1629178166, 'blockNumber': 1629178166, 'action': 'deposit', 'actionData': {'type': 'Deposit', 'amount': '2000000000', 'assetSymbol': 'USDC', 'assetPriceUSD': '0.9938318274296357543568636362026045', 'poolId': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174', 'userId': '0x00000000001accfa9cef68cf5371a23025b6d4b6'}, '__v': 0, 'createdAt': {'$date': '2025-05-08T23:06:39.465Z'}, 'updatedAt': {'$date': '2025-05-08T23:06:39.465Z'}}
